In [63]:
import pandas as pd
import numpy as np
from datetime import *
from plotnine import *
import statsmodels.api as sm

In [16]:
df = pd.concat([pd.read_csv("data/lichess_swiss_rating_histories_1.csv",parse_dates=['date']),
                pd.read_csv("data/lichess_swiss_rating_histories_2.csv",parse_dates=['date'])])
print(df.shape)
df.head()

(8623368, 4)


,user_id,time_control,date,rating
0,0,Blitz,2019-01-04,2244
1,0,Blitz,2019-01-05,2256
2,0,Blitz,2019-01-06,2231
3,0,Blitz,2019-01-08,2307
4,0,Blitz,2019-01-29,2288


In [5]:
df['user_id'].nunique()

17171

In [6]:
df['date'].max()

'2021-08-03'

In [11]:
# The latest date we have data on
max_outcome_date = df['date'].max()
# The latest date that can be used for training to ensure we'll always have 2 years in advance of outcomes data
max_training_date = max_outcome_date - timedelta(days=365*2)
max_outcome_date,max_training_date

(Timestamp('2021-08-03 00:00:00'), Timestamp('2019-08-04 00:00:00'))

In [13]:
# The earliest date we have data on
minn_training_date = df['date'].min()
minn_training_date

Timestamp('2019-01-01 00:00:00')

In [53]:
# The latest ratings that can be used for training
df_training = df.query('date<=@max_training_date')
df_outcomes = df.query('date>@max_training_date')
latest_training_ratings = df_training.sort_values("date",ascending=False).drop_duplicates(['user_id','time_control'])
latest_training_ratings.sample(5)

,user_id,time_control,date,rating
637431,11342,Bullet,2019-08-02,2113
2948792,15641,Bullet,2019-08-04,1273
1142598,2667,Rapid,2019-08-03,1518
1083190,2533,Classical,2019-07-23,1282
2248754,14434,Blitz,2019-08-04,1840


In [29]:
# Ratings from X days before the max training date
max_training_date_minus_30 = max_training_date-timedelta(days=30)
max_training_date_minus_90 = max_training_date-timedelta(days=90)
max_training_date_minus_180 = max_training_date-timedelta(days=180)
hist_ratings_30 = df.query('date<=@max_training_date_minus_30').sort_values("date",ascending=False).drop_duplicates(['user_id','time_control'])
hist_ratings_90 = df.query('date<=@max_training_date_minus_90').sort_values("date",ascending=False).drop_duplicates(['user_id','time_control'])
hist_ratings_180 = df.query('date<=@max_training_date_minus_180').sort_values("date",ascending=False).drop_duplicates(['user_id','time_control'])
hist_ratings_180.head()

,user_id,time_control,date,rating
7033,10091,Blitz,2019-02-05,1106
1488095,13023,Rapid,2019-02-05,1339
1475936,12996,Bullet,2019-02-05,1722
1476181,12996,Rapid,2019-02-05,1812
1476463,12997,Bullet,2019-02-05,1454


In [30]:
# Peak ratings
hist_ratings_peak = df_training.sort_values("rating",ascending=False).drop_duplicates(['user_id','time_control'])
hist_ratings_peak.head()

,user_id,time_control,date,rating
3141002,6935,Bullet,2019-03-26,2872
4478689,9624,Bullet,2019-07-21,2852
2810666,6252,Bullet,2019-06-26,2847
4478286,9624,Blitz,2019-08-04,2833
3039334,6727,Bullet,2019-05-05,2832


In [101]:
# Add features to base table
df_base = latest_training_ratings.merge(hist_ratings_30[['user_id','time_control','rating']],
                how='left',on=['user_id','time_control'],suffixes=['_latest','_30']).merge(
            hist_ratings_90[['user_id','time_control','rating']],
                how='left',on=['user_id','time_control']).merge(
            hist_ratings_180[['user_id','time_control','rating']],
                how='left',on=['user_id','time_control'],suffixes=['_90','_180']).merge(
            hist_ratings_peak[['user_id','time_control','rating']].rename(columns={'rating':'rating_peak'}),
                how='left',on=['user_id','time_control'])
df_base['rating_30_diff'] = df_base['rating_latest']-df_base['rating_30']
df_base['rating_90_diff'] = df_base['rating_latest']-df_base['rating_90']
df_base['rating_180_diff'] = df_base['rating_latest']-df_base['rating_180']
df_base['rating_peak_diff'] = df_base['rating_latest']-df_base['rating_peak']
print(df_base.shape)
df_base.sample(10)

(41530, 12)


,user_id,time_control,date,rating_latest,rating_30,rating_90,rating_180,rating_peak,rating_30_diff,rating_90_diff,rating_180_diff,rating_peak_diff
21237,10465,Blitz,2019-07-15,1763,1777.0,1786.0,1786.0,1786,-14.0,-23.0,-23.0,-23
29003,15181,Rapid,2019-06-07,1495,1495.0,1435.0,1344.0,1495,0.0,60.0,151.0,0
30471,10572,Rapid,2019-05-26,1303,1303.0,NaN,NaN,1432,0.0,NaN,NaN,-129
11300,17210,Blitz,2019-08-02,1722,1711.0,1603.0,1540.0,1722,11.0,119.0,182.0,0
19181,6712,Blitz,2019-07-21,2163,2210.0,2069.0,2172.0,2232,-47.0,94.0,-9.0,-69
8654,9882,Classical,2019-08-03,1748,1735.0,1599.0,NaN,1748,13.0,149.0,NaN,0
3675,434,Bullet,2019-08-04,2160,2280.0,2204.0,2122.0,2342,-120.0,-44.0,38.0,-182
33049,5950,Classical,2019-05-03,1787,1787.0,1787.0,NaN,1787,0.0,0.0,NaN,0
34988,7460,Rapid,2019-04-13,963,963.0,963.0,1157.0,1157,0.0,0.0,-194.0,-194
13527,5151,Blitz,2019-07-31,1680,1749.0,1732.0,1684.0,1802,-69.0,-52.0,-4.0,-122


In [103]:
# Filter to people who have played rated games in the time control before 30 days ago...
# ... and have played at least one rated game in the time control within the last 30 days
df_base = df_base[(df_base['rating_30'].notna())&(df_base['date']>=max_training_date_minus_30)]
df_base.shape

(21641, 12)

In [104]:
# 100 rating point gain model
df_100 = latest_training_ratings.copy()
df_100['target_rating'] = df_100['rating'] + 100
df_100 = df_100.merge(df_outcomes,on=['user_id','time_control'],how='outer',suffixes=['_current','_future'])
df_100.head()

,user_id,time_control,date_current,rating_current,target_rating,date_future,rating_future
0,4250,Blitz,2019-08-04,1615.0,1715.0,2019-08-05,1598.0
1,4250,Blitz,2019-08-04,1615.0,1715.0,2019-08-06,1630.0
2,4250,Blitz,2019-08-04,1615.0,1715.0,2019-08-07,1626.0
3,4250,Blitz,2019-08-04,1615.0,1715.0,2019-08-08,1654.0
4,4250,Blitz,2019-08-04,1615.0,1715.0,2019-08-09,1644.0


In [105]:
# Successes - filter to where future rating >= target rating, then take earliest date for each user/time control
df_100_1 = df_100.query('rating_future>=target_rating').sort_values("date_future").drop_duplicates(['user_id','time_control'])
print(df_100_1.shape)
df_100_1.sample(5)

(34143, 7)


,user_id,time_control,date_current,rating_current,target_rating,date_future,rating_future
3496505,15260,Blitz,2019-07-31,1888.0,1988.0,2019-12-20,2004.0
5706914,13024,Rapid,2019-04-22,1450.0,1550.0,2020-04-18,1557.0
5911299,1314,Rapid,2019-03-14,1200.0,1300.0,2020-04-05,1314.0
2430970,1093,Classical,2019-08-03,1492.0,1592.0,2020-09-10,1600.0
6044788,12490,Blitz,2019-02-07,1198.0,1298.0,2020-07-02,1365.0


In [106]:
# Successes and failures 
df_100 = df_base.merge(df_100_1,on=['user_id','time_control'],how='left').drop(['rating_current','rating_future','date_current'],axis=1)
# Was the target rating achieved?
df_100['y_bin'] = df_100['date_future'].notna().astype(int)
# If so, when?
df_100['y_cont'] = (df_100['date_future']-max_training_date).dt.days
print(df_100.shape)
df_100.sample(10)

(21641, 16)


,user_id,time_control,date,rating_latest,rating_30,rating_90,rating_180,rating_peak,rating_30_diff,rating_90_diff,rating_180_diff,rating_peak_diff,target_rating,date_future,y_bin,y_cont
1163,9403,Rapid,2019-08-04,1779,1679.0,NaN,NaN,1779,100.0,NaN,NaN,0,1879.0,2019-08-21,1,17.0
3207,3337,Blitz,2019-08-04,1752,1700.0,1855.0,1664.0,1855,52.0,-103.0,88.0,-103,1852.0,2019-11-20,1,108.0
7190,6236,Bullet,2019-08-03,1888,1950.0,1894.0,1816.0,2009,-62.0,-6.0,72.0,-121,1988.0,2019-11-07,1,95.0
16319,737,Rapid,2019-07-24,1358,1335.0,1335.0,NaN,1367,23.0,23.0,NaN,-9,1458.0,2021-07-08,1,704.0
14423,7661,Blitz,2019-07-28,1602,1641.0,1644.0,1681.0,1764,-39.0,-42.0,-79.0,-162,1702.0,2020-04-15,1,255.0
12627,12878,Blitz,2019-07-31,1265,1369.0,NaN,NaN,1419,-104.0,NaN,NaN,-154,1365.0,2019-08-20,1,16.0
9894,8669,Bullet,2019-08-02,1301,1306.0,1329.0,NaN,1358,-5.0,-28.0,NaN,-57,1401.0,2020-02-23,1,203.0
16425,6908,Classical,2019-07-24,2010,2014.0,NaN,NaN,2014,-4.0,NaN,NaN,-4,2110.0,2019-11-07,1,95.0
7731,14175,Blitz,2019-08-03,1754,1653.0,1710.0,NaN,1893,101.0,44.0,NaN,-139,1854.0,2019-12-28,1,146.0
6706,8714,Blitz,2019-08-03,1196,1075.0,988.0,NaN,1386,121.0,208.0,NaN,-190,1296.0,2019-10-27,1,84.0


In [107]:
# Feature engineering
df_100 = pd.get_dummies(df_100,columns=['time_control'])
df_100.head()

,user_id,date,rating_latest,rating_30,rating_90,rating_180,rating_peak,rating_30_diff,rating_90_diff,rating_180_diff,rating_peak_diff,target_rating,date_future,y_bin,y_cont,time_control_Blitz,time_control_Bullet,time_control_Classical,time_control_Rapid
0,4250,2019-08-04,1615,1505.0,1525.0,1555.0,1636,110.0,90.0,60.0,-21,1715.0,2019-09-06,1,33.0,1,0,0,0
1,10683,2019-08-04,1216,1109.0,1031.0,NaN,1216,107.0,185.0,NaN,0,1316.0,2019-12-05,1,123.0,1,0,0,0
2,10440,2019-08-04,1433,1522.0,1414.0,1234.0,1598,-89.0,19.0,199.0,-165,1533.0,2019-08-31,1,27.0,1,0,0,0
3,10837,2019-08-04,1604,1606.0,NaN,NaN,1761,-2.0,NaN,NaN,-157,1704.0,2020-04-12,1,252.0,0,1,0,0
4,15733,2019-08-04,1156,1253.0,NaN,NaN,1253,-97.0,NaN,NaN,-97,1256.0,2019-09-03,1,30.0,1,0,0,0


In [112]:
predictors = ['rating_latest','time_control_Blitz','time_control_Bullet','time_control_Rapid','rating_peak_diff','rating_30_diff']
df_100_predictors = sm.add_constant(df_100[predictors])
logit_100 = sm.Logit(endog=df_100['y_bin'],exog=df_100_predictors).fit()
logit_100.summary()

Optimization terminated successfully.
         Current function value: 0.303149
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  y_bin   No. Observations:                21641
Model:                          Logit   Df Residuals:                    21634
Method:                           MLE   Df Model:                            6
Date:                Tue, 03 Aug 2021   Pseudo R-squ.:                 0.08724
Time:                        17:27:38   Log-Likelihood:                -6560.5
converged:                       True   LL-Null:                       -7187.5
Covariance Type:            nonrobust   LLR p-value:                9.708e-268
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   1.7820      0.142     12.593      0.000       1.505       2.059
rating_latest          -0.0007   7.68e-05     -8.774      0.000      -0.001      -0.001
time_control_Blitz      1.9571      0.065     29.933      0.000       1.829       2.085
time_control_Bullet     1.4601      0.065     22.603      0.000       1.334       1.587
time_control_Rapid      1.2828      0.066     19.295      0.000       1.152       1.413
rating_peak_diff       -0.0034      0.000     -8.190      0.000      -0.004      -0.003
rating_30_diff         -0.0006      0.000     -1.592      0.111      -0.001       0.000
=======================================================================================
"""

In [117]:
# Regression
df_100_reg = df_100[df_100['y_bin']==1]
df_100_reg_predictors = sm.add_constant(df_100_reg[predictors])
ols_100 = sm.OLS(endog=df_100_reg['y_cont'],exog=df_100_reg_predictors).fit()
ols_100.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 y_cont   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.080
Method:                 Least Squares   F-statistic:                     282.3
Date:                Tue, 03 Aug 2021   Prob (F-statistic):               0.00
Time:                        17:30:37   Log-Likelihood:            -1.2693e+05
No. Observations:               19407   AIC:                         2.539e+05
Df Residuals:                   19400   BIC:                         2.539e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 286.4500      7.765     36.888      0.000     271.229     301.671
rating_latest          -0.0219      0.004     -5.676      0.000      -0.029      -0.014
time_control_Blitz    -36.3767      4.477     -8.125      0.000     -45.152     -27.602
time_control_Bullet   -54.6906      4.648    -11.768      0.000     -63.800     -45.581
time_control_Rapid     39.7287      4.865      8.166      0.000      30.193      49.265
rating_peak_diff        0.4194      0.017     24.563      0.000       0.386       0.453
rating_30_diff         -0.0497      0.019     -2.656      0.008      -0.086      -0.013
==============================================================================
Omnibus:                     3474.703   Durbin-Watson:                   1.921
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5691.137
Skew:                           1.232   Prob(JB):                         0.00
Kurtosis:                       3.983   Cond. No.                     1.27e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.27e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [109]:
df_100['prob'] = logit_100.predict(exog=df_100_predictors)
df_100['prob'].head()

0    0.934686
1    0.945811
2    0.967366
3    0.937059
4    0.966018
Name: prob, dtype: float64

In [110]:
df_100['prob'].mean()

0.8967700198696923

In [111]:
df_100['y_bin'].mean()

0.8967700198696917

## Features
- Target time control (likely interacted with various other features)
- Current rating (likely nonlinear relationship)
- Rating growth in last 30 days / 90 days / 180 days
- Rating volatility measures
- Peak historical rating relative to current rating
- Rating in other time controls + puzzles
- Rating growth in other time controls + puzzles
- Difference between other time control ratings + target time control rating
- How long you've been on lichess
- How many games you've played (ever, and within last 30 days, and within the target time control - if you haven't played many it could mean more uncertainty). Consider that most discord bot users will have played more recent rated games in the target time control than the typical user in the training data. 
- Last time you played a rated game in the target time control (if it's a long time ago, it could mean more uncertainty)

## Outcomes
- Will you ever achieve a rating that's X rating points higher than your current rating in the next Y months (X is calculated from target rating submitted by user, Y = 24?)
- If so, when will you first reach the target rating? (point estimate + prediction interval of dates) - use number of days as outcome, then transform to date for the bot message

## Notes:
- Might want to train multiple models for various values of X between 0 and 500 (with multiples of 10 to speed up, maybe, but smaller multiples for low values of X)
- Might want to exclude certain accounts (if they stop playing on lichess or in that time control recently, if they are very new, if something else is weird)
- Use cross-validation since sample size might be constrained
- Need to impute nulls
- Need to write code for scoring based on discord input (including lichess API querying)
- Need to figure out how to make prediction interval
- Add more comments + documentation to final version
